# Qualitative Evaluation: Diff-Analyse

Hinweis: In [JupyterLab](http://localhost:8891/doc/tree/qualitative_evaluation_json_issues.ipynb) öffnen (Light Theme), damit die Diffs nebeneinander angezeigt werden.

In [38]:
import difflib
import json
import re
from itertools import combinations

from IPython.display import HTML, Markdown, display

display(HTML("<style>.container { width:100% !important; }</style>"))


# Source: https://stackoverflow.com/a/65417026
class MyHTML(difflib.HtmlDiff):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)

        # append new styles inside new class
        self._styles = (
            "table.diff {font-family:Courier; border:medium; text-align:left}"
            ".diff_header {background-color:#e0e0e0; text-align:left}"
            "td.left {text-align:left}"
            ".diff_next {background-color:#c0c0c0; text-align:left}"
            ".diff_add {background-color:#aaffaa; text-align:left}"
            ".diff_chg {background-color:#ffff77; text-align:left}"
            ".diff_sub {background-color:#ffaaaa; text-align:left}"
        )

    # function from source code - I removed only `nowrap="nowrap"`
    def _format_line(self, side, flag, linenum, text):
        try:
            linenum = "%d" % linenum
            id = ' id="%s%s"' % (self._prefix[side], linenum)
        except TypeError:
            # handle blank lines where linenum is '>' or ''
            id = ""

        # replace those things that would get confused with HTML symbols
        text = text.replace("&", "&amp;").replace(">", "&gt;").replace("<", "&lt;")

        # make space non-breakable so they don't get compressed or line wrapped
        text = text.replace(" ", "&nbsp;").rstrip()

        # vvv ---- removed `nowrap="nowrap"` --- vvv
        return '<td class="diff_header"%s>%s</td><td>%s</td>' % (id, linenum, text)


def show_diff(combs, result_json_dict):
    for key1, key2 in combs:
        display(Markdown("# {} - {}".format(key1, key2)))
        for idx, (value1, value2) in enumerate(
            zip(result_json_dict[key1], result_json_dict[key2])
        ):
            diff = MyHTML(wrapcolumn=130)
            diff = diff.make_file(
                re.sub(r"\s{2,}", " ", value1)
                .replace("\n", " ")
                .replace(".", ".\n\n")
                .split("\n\n"),
                re.sub(r"\s{2,}", " ", value2)
                .replace("\n", " ")
                .replace(".", ".\n\n")
                .split("\n\n"),
                key1,
                key2,
                context=True,
                numlines=1,
            )
            display(Markdown("### Doc #{}".format(idx)))
            display(HTML(diff.replace("<td>", '<td class="left">')))

In [39]:
with open("qualitative_evaluation_0_results_2023-12-04T15_30_39.json", "r") as f:
    result_json_dict = json.loads(f.read())

# Vergleich spaCy-Modelle

In [40]:
combs = list(
    combinations([k for k in result_json_dict.keys() if k.startswith("de_")], 2)
)

In [41]:
show_diff(combs, result_json_dict)

# de_core_news_md - de_core_news_lg

### Doc #0

### Doc #1

### Doc #2

# Vergleich SpanMarker-Modelle

In [42]:
combs = list(
    combinations([k for k in result_json_dict.keys() if k.startswith("sm")], 2)
)

In [43]:
show_diff(combs, result_json_dict)

# sm_gelectra - sm_teams

### Doc #0

### Doc #1

### Doc #2

# sm_gelectra - sm_bert

...

### Doc #2

# sm_bert - sm_td_bert

### Doc #0

### Doc #1

### Doc #2

# Finaler Vergleich sm_gelectra, fl_german_large, de_core_news_lg, swissbert

In [44]:
combs = list(
    combinations(["sm_gelectra", "fl_german_large", "de_core_news_lg", "swissbert"], 2)
)
combs

[('sm_gelectra', 'fl_german_large'),
 ('sm_gelectra', 'de_core_news_lg'),
 ('sm_gelectra', 'swissbert'),
 ('fl_german_large', 'de_core_news_lg'),
 ('fl_german_large', 'swissbert'),
 ('de_core_news_lg', 'swissbert')]

In [45]:
show_diff(combs, result_json_dict)

# sm_gelectra - fl_german_large

### Doc #0

### Doc #1

### Doc #2

# sm_gelectra - de_core_news_lg

...

### Doc #2

# de_core_news_lg - swissbert

### Doc #0

### Doc #1

### Doc #2